#### Analyzing sar data programatically:

In [1]:
import os
import sys
parentDir = os.path.abspath('..')
sys.path.append(parentDir)

In [2]:
from sar import sar
from sar.utils import Table
from sar.utils import BarPlot
from sar.utils import printList, getListDisplayText, printStreamBuffer, stack, underline
from sar.utils import Stage
import json

In [3]:
import glob
sarfiles = glob.glob( './sar*txt')

In [4]:
sarfiles

['./saru.txt',
 './sarb.txt',
 './sarw.txt',
 './sarr.txt',
 './sar-A.txt',
 './sar.txt']

In [5]:
fileDictionary={}
for fileName in sarfiles:
    with open (fileName) as fhand:
        fileDictionary[fileName]=sar(fhand)

In [6]:
fileDictionary

{'./saru.txt': <sar.sar.sar at 0x104147f50>,
 './sarb.txt': <sar.sar.sar at 0x104147dd0>,
 './sarw.txt': <sar.sar.sar at 0x10416fef0>,
 './sarr.txt': <sar.sar.sar at 0x10542cec0>,
 './sar-A.txt': <sar.sar.sar at 0x10542cef0>,
 './sar.txt': <sar.sar.sar at 0x10444e0c0>}

In [7]:
[(n, s.fetchtables('%user')) for n,s in fileDictionary.items()]

[('./saru.txt', []),
 ('./sarb.txt', []),
 ('./sarw.txt', []),
 ('./sarr.txt', []),
 ('./sar-A.txt', []),
 ('./sar.txt', [<sar.utils.table.Table at 0x105b86300>])]

In [8]:
[(n, s.fetchtables('%usr')) for n,s in fileDictionary.items()]

[('./saru.txt', [<sar.utils.table.Table at 0x105b86870>]),
 ('./sarb.txt', []),
 ('./sarw.txt', []),
 ('./sarr.txt', []),
 ('./sar-A.txt', [<sar.utils.table.Table at 0x105b866c0>]),
 ('./sar.txt', [])]

In [9]:
s1=fileDictionary['./sar.txt']
s1.tableList

[{428484498971099520: <sar.utils.table.Table at 0x105b7f470>},
 {428484498971099520: <sar.utils.table.Table at 0x105b85af0>}]

In [10]:
for t in s1.fetchtables('%user'):
    t.print()

Timestamp       CPU     %user     %nice     %system     %iowait     %steal     %idle     
---------       ---     -----     -----     -------     -------     ------     -----     
12:11:00 AM     all     0.04      0.02      0.18        0.05        0.00       99.72     
12:20:08 AM     all     0.01      0.00      0.12        0.02        0.00       99.85     
12:30:15 AM     all     0.09      0.00      0.23        0.02        0.00       99.65     
12:40:07 AM     all     0.05      0.00      0.19        0.02        0.00       99.74     
12:50:15 AM     all     0.01      0.00      0.11        0.02        0.00       99.86     
01:00:15 AM     all     0.02      0.00      0.11        0.02        0.00       99.85     
01:10:08 AM     all     0.01      0.00      0.10        0.02        0.00       99.87     
04:02:49 AM     all     0.01      0.00      0.11        0.02        0.00       99.87     
04:36:35 AM     all     0.04      0.01      0.14        0.04        0.00       99.78     
04:46:50 A

In [11]:
def analyzeFields(fields, sarDataInstance):
    """analyzeFields runs once for every report.
    Each json field maps to a report name using "report" key. If a field specifies no report name, it will default to "Others"
    At this time, analyzeFields performs a sorting of each field referenced in sar.json using "sort" key.
    Fields are sorted in descending order - presuming the highest values are the ones of interest.
    A group column is displayed when specified in sar.json using "group" key."""
    lists=[]
    for field in fields:
        sort=field.get('sort')
        tables=sarDataInstance.fetchtables(sort)

        for t in tables:
            data=BarPlot.appendBarPlot(t.get(('Timestamp',sort),filterFunc=lambda x:True, sortKey=lambda x:float(x(sort)), reverse=True)[:5],1)
            lists.append(getListDisplayText(header=('Timestamp',sort,''), data=data))
    return lists

In [12]:
with open('../conf/sar.json','r') as jsontst:
    jsonfields=json.load(jsontst)

In [13]:
reportsDict={}
for d in jsonfields['fields']:
    reportName=d.get('report','Others')
    reportsDict[reportName]=reportsDict.get(reportName,[])+[d]

In [14]:
for n,s in fileDictionary.items():
    print(f'Analyzing: {n}:')
    for report in reportsDict.items():
        lists=analyzeFields(report[1], s)
        if len(lists) > 0:
            print(Stage('Analyzing '+report[0]).stringValue())
            stack(*lists, padding=20, lineCapacity=200)

Analyzing: ./saru.txt:

=          Analyzing Memory       =

Timestamp     %memused                                     Timestamp     kbmemused                                 Timestamp     %commit                                     
---------     --------                                     ---------     ---------                                 ---------     -------                                     
19:35:01      78.27        ▄▄▄▄▄▄▄                         19:35:01      38380936      ▄▄▄                         23:45:01      70.07       ▄▄▄▄▄▄▄                         
19:30:01      78.18        ▄▄▄▄▄▄▄                         19:30:01      38337740      ▄▄▄                         23:55:01      70.07       ▄▄▄▄▄▄▄                         
19:25:01      78.12        ▄▄▄▄▄▄▄                         19:25:01      38306944      ▄▄▄                         23:25:01      70.06       ▄▄▄▄▄▄▄                         
19:20:01      78.02        ▄▄▄▄▄▄▄                         19:20:01  

In [15]:
with open('sar.txt') as fhand:
    sarObj=sar(fhand)


In [16]:
sarObj.printRawSar()

Linux 6.8.0-50-generic (ubuntu2404) 	12/15/2024 	_aarch64_	(2 CPU)

12:01:11 AM     CPU     %user     %nice   %system   %iowait    %steal     %idle
12:11:00 AM     all      0.04      0.02      0.18      0.05      0.00     99.72
12:20:08 AM     all      0.01      0.00      0.12      0.02      0.00     99.85
12:30:15 AM     all      0.09      0.00      0.23      0.02      0.00     99.65
12:40:07 AM     all      0.05      0.00      0.19      0.02      0.00     99.74
12:50:15 AM     all      0.01      0.00      0.11      0.02      0.00     99.86
01:00:15 AM     all      0.02      0.00      0.11      0.02      0.00     99.85
01:10:08 AM     all      0.01      0.00      0.10      0.02      0.00     99.87
04:02:49 AM     all      0.01      0.00      0.11      0.02      0.00     99.87
04:36:35 AM     all      0.04      0.01      0.14      0.04      0.00     99.78
04:46:50 AM     all      0.04      0.00      0.11      0.02      0.00     99.83
Average:        all      0.03      0.00      0.14   

In [17]:
sarObj.query(lambda f:float(f('%system')) > 0.1)


Timestamp       CPU     %user     %nice     %system     %iowait     %steal     %idle     
---------       ---     -----     -----     -------     -------     ------     -----     
12:11:00 AM     all     0.04      0.02      0.18        0.05        0.00       99.72     
12:20:08 AM     all     0.01      0.00      0.12        0.02        0.00       99.85     
12:30:15 AM     all     0.09      0.00      0.23        0.02        0.00       99.65     
12:40:07 AM     all     0.05      0.00      0.19        0.02        0.00       99.74     
12:50:15 AM     all     0.01      0.00      0.11        0.02        0.00       99.86     
01:00:15 AM     all     0.02      0.00      0.11        0.02        0.00       99.85     
04:02:49 AM     all     0.01      0.00      0.11        0.02        0.00       99.87     
04:36:35 AM     all     0.04      0.01      0.14        0.04        0.00       99.78     
04:46:50 AM     all     0.04      0.00      0.11        0.02        0.00       99.83     
05:10:02 

In [18]:
with open('saru.txt') as fhand:
    sar(fhand).query(lambda f:f('DEV')=='dev8-0' and f('Timestamp').split(':')[0]=='00')


Timestamp     DEV        tps       rd_sec/s     wr_sec/s     avgrq-sz     avgqu-sz     await      svctm     %util     
---------     ---        ---       --------     --------     --------     --------     -----      -----     -----     
00:05:31      dev8-0     66.14     388.19       11574.61     180.88       23.65        357.61     3.95      26.09     
00:10:01      dev8-0     2.58      3.32         40.79        17.11        0.00         0.53       0.46      0.12      
00:15:01      dev8-0     2.52      2.99         39.00        16.69        0.00         0.40       0.39      0.10      
00:20:01      dev8-0     2.48      2.99         39.20        17.01        0.00         0.76       0.47      0.12      
00:25:01      dev8-0     2.54      3.00         38.96        16.53        0.00         0.63       0.41      0.10      
00:30:01      dev8-0     2.56      2.99         38.69        16.28        0.00         0.52       0.42      0.11      
00:35:02      dev8-0     2.42      3.00        

In [19]:
for table in sarObj.fetchtables('%user'):
    table.print()

Timestamp       CPU     %user     %nice     %system     %iowait     %steal     %idle     
---------       ---     -----     -----     -------     -------     ------     -----     
12:11:00 AM     all     0.04      0.02      0.18        0.05        0.00       99.72     
12:20:08 AM     all     0.01      0.00      0.12        0.02        0.00       99.85     
12:30:15 AM     all     0.09      0.00      0.23        0.02        0.00       99.65     
12:40:07 AM     all     0.05      0.00      0.19        0.02        0.00       99.74     
12:50:15 AM     all     0.01      0.00      0.11        0.02        0.00       99.86     
01:00:15 AM     all     0.02      0.00      0.11        0.02        0.00       99.85     
01:10:08 AM     all     0.01      0.00      0.10        0.02        0.00       99.87     
04:02:49 AM     all     0.01      0.00      0.11        0.02        0.00       99.87     
04:36:35 AM     all     0.04      0.01      0.14        0.04        0.00       99.78     
04:46:50 A